In [ ]:
import os
import tensorflow as tf
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
stimulus = np.load('/Users/iuliarusu/Documents/Sharpee/ProcAiryData/inpfull_op50_SF.npz')
Y_full = np.load('/Users/iuliarusu/Documents/Sharpee/ProcAiryData/Yfull_op50_SF.npz')

In [ ]:
stimulus_concat = np.concatenate([stimulus['arr_0'], stimulus['arr_1'], stimulus['arr_2'], stimulus['arr_3'], stimulus['arr_4'], stimulus['arr_5'], stimulus['arr_6']], axis =1)

In [ ]:
stimulus_concat = stimulus_concat.T

In [ ]:
#import AVA data from all cells
AVA_0_df = pd.read_csv ('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_0_df.csv')
AVA_1_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_1_df.csv') 
AVA_2_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_2_df.csv')
AVA_3_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_3_df.csv') 
AVA_4_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_4_df.csv') 
AVA_5_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_5_df.csv') 
AVA_6_df = pd.read_csv('/Users/iuliarusu/Documents/Sharpee/Clustering/clustered_bacterial_stim/AVA_6_df.csv')

In [ ]:
AVA_0 = np.array(AVA_0_df.iloc[:, 1: -2])
AVA_1 = np.array(AVA_1_df.iloc[:, 1: -3])
AVA_2 = np.array(AVA_2_df.iloc[:, 1: -3])
AVA_3 = np.array(AVA_3_df.iloc[:, 1: -3])
AVA_4 = np.array(AVA_4_df.iloc[:, 1: -3])
AVA_5 = np.array(AVA_5_df.iloc[:, 1: -3])
AVA_6 = np.array(AVA_6_df.iloc[:, 1: -3])

In [ ]:
AVA_all = np.concatenate([AVA_0, AVA_1, AVA_2, AVA_3, AVA_4, AVA_5, AVA_6], axis = 0)

In [ ]:
#seperate stim 1 and stim 0, 
AVA_stim_0 = np.concatenate([AVA_0, AVA_1, AVA_6], axis = 0)
AVA_stim_1 = np.concatenate([AVA_2, AVA_3, AVA_4, AVA_5], axis = 0)

In [ ]:
def get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins=10, nsamples=10):
    response_vector = []
    stimulus_vector = []
    for _ in range(nsamples):
        r = response[cell_idx, start_idx: start_idx + bin_size]
        response_vector.append(np.average(r))
        print('response', response_vector)

        bin_stimuli = []
        for i in range(num_bins):
           
            end_idx = (start_idx ) - (i * bin_size)
            print('start_idx', start_idx)
            print('end', end_idx)
            start_idx_bin = end_idx + bin_size
            print('start_idx_bin', start_idx_bin)

            # Check if the calculated index is within the bounds
            if start_idx_bin < 0:
                break

            # Append the average of stimulus over the bin
            bin_stimuli.append(np.average(stimulus[0, end_idx:start_idx_bin]))
            print('stim', stimulus[0, start_idx_bin:end_idx])
            print('bin_stimuli', bin_stimuli)

        # Collect all averages per sample
        stimulus_vector.append(bin_stimuli)
        print('stimulus_vector', stimulus_vector)

        # Update start index for the next sample
        start_idx += bin_size

    # Convert list of lists into a 2D NumPy array
    stimulus_matrix = np.array(stimulus_vector)  # Transpose to match expected dimensions

    return stimulus_matrix, np.array(response_vector)

In [ ]:
stim_0 = stimulus_concat[0, :].reshape(1600, 1)
stim_1 = stimulus_concat[2, :].reshape(1600, 1)


In [ ]:
all_data_stim0 = []
all_labels_stim0 = []

bin_size = 10
start_idx = 200  # Starting index for the stimulus and response extraction
num_bins = 10
nsamples = 100 #amount of times you're moving over the data

stimulus = stim_0.T
response = AVA_stim_0
cell_idx = 0 #cell in response
# Calculate data and labels for each cell or half the cells
num_cells = response.shape[0]  # or response.shape[0] // 2 for half of the cells
for cell_idx in range(num_cells):
    data, label = get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins, nsamples)
    all_data_stim0.append(data)
    all_labels_stim0.append(label)

In [ ]:
all_data__stim0_array = np.concatenate(all_data_stim0, axis=0)

# Convert list of 1D arrays to a single 2D array (if all_labels consists of 1D arrays)
all_labels__stim0_array = np.concatenate(all_labels_stim0, axis=0)

In [ ]:
all_data_stim1 = []
all_labels_stim1 = []

bin_size = 10
start_idx = 200  # Starting index for the stimulus and response extraction
num_bins = 10
nsamples = 100 #amount of times you're moving over the data

stimulus = stim_1.T
response = AVA_stim_1
cell_idx = 0 #cell in response
# Calculate data and labels for each cell or half the cells
num_cells = response.shape[0]  # or response.shape[0] // 2 for half of the cells
for cell_idx in range(num_cells):
    data, label = get_bins_avg(stimulus, response, bin_size, start_idx, cell_idx, num_bins, nsamples)
    all_data_stim1.append(data)
    all_labels_stim1.append(label)

In [ ]:
all_data__stim1_array = np.concatenate(all_data_stim1, axis=0)

# Convert list of 1D arrays to a single 2D array (if all_labels consists of 1D arrays)
all_labels__stim1_array = np.concatenate(all_labels_stim1, axis=0)

In [ ]:
#concatenate both stimuli
all_data_array = np.concatenate([all_data__stim0_array, all_data__stim1_array], axis = 0)
all_labels_array = np.concatenate([all_labels__stim0_array, all_labels__stim1_array], axis = 0) 

In [ ]:
S, Y = all_data_array, all_labels_array 

In [ ]:
D = 10

In [ ]:
def make_quad_model(D, optimizer):

    input_layer = keras.layers.Input((D,))

    linear_layer = keras.layers.Dense(1)(input_layer)

    quadratic_layer = keras.layers.Dense(D,use_bias=False)(input_layer)

    multiply_layer = keras.layers.Multiply()([quadratic_layer,input_layer])

    ones_layer = keras.layers.Dense(1,use_bias=False,trainable=False,kernel_initializer='ones')(multiply_layer)

    add_layer = keras.layers.Add()([linear_layer,ones_layer])

    output_layer = keras.layers.Activation('sigmoid')(add_layer)

    model = keras.models.Model(input_layer,output_layer)

    model.compile(optimizer= optimizer, loss='binary_crossentropy')

    return model

In [ ]:
def run_jackknife_models(S, Y):

    patience = 3 # early stopping
    epochs = 1000 # max number of cycles to train
    all_best_weights = []

    sq1, sq2, sq3, sq4 = np.array_split(S, 4)
    yq1, yq2, yq3, yq4 = np.array_split(Y, 4)

    j1 = {
    'S_train': np.concatenate((sq2, sq3, sq4), axis=0),
    'S_valid': sq1,
    'Y_train': np.concatenate((yq2, yq3, yq4), axis=0),
    'Y_valid': yq1
}
    j2 = {
    'S_train': np.concatenate((sq1, sq3, sq4), axis=0),
    'S_valid': sq2,
    'Y_train': np.concatenate((yq1, yq3, yq4), axis=0),
    'Y_valid': yq2
}
    j3 = {
    'S_train': np.concatenate((sq1, sq2, sq4), axis=0),
    'S_valid': sq3,
    'Y_train': np.concatenate((yq1, yq2, yq4), axis=0),
    'Y_valid': yq3
}
    j4 = {
    'S_train': np.concatenate((sq1, sq2, sq3), axis=0),
    'S_valid': sq4,
    'Y_train': np.concatenate((yq1, yq2, yq3), axis=0),
    'Y_valid': yq4
}

    jackknives = [j1, j2, j3, j4]

    for i, jackknife in enumerate(jackknives):
        S_train = jackknife['S_train']
        S_valid = jackknife['S_valid']
        Y_train = jackknife['Y_train']
        Y_valid = jackknife['Y_valid']

 
        adam_optimizer = Adam(learning_rate=0.001)
        best_weights = []
        
    # Creating the model with the optimizer passed as an argument
        model = make_quad_model(S_train.shape[1], adam_optimizer)

    # Setting up callbacks
        filepath = f'best_weights_{i}.weights.h5' 
        callbacks = [
                EarlyStopping(patience=patience, min_delta=1e-5, monitor='val_loss', verbose=1),
                ModelCheckpoint(filepath=filepath, save_weights_only=True, save_best_only=True, verbose=1)
        ]

    # Training the model
        H = model.fit(S_train, Y_train, epochs=epochs, callbacks=callbacks, validation_data=(S_valid, Y_valid), verbose=2)
        print(model.summary())
        model.load_weights(filepath)#loads the best weights from the training
        best_weights.append(model.get_weights()) #retreives best weights from load_weights callback
    
        all_best_weights.append(best_weights)

    return all_best_weights

In [ ]:
def run_models(S, Y, num_models=4, global_seed = 24):
    np.random.seed(global_seed)
    test_size = 0.25
    patience = 3
    epochs = 1000
    all_best_weights = []
    
    
    
    
    for i in range(num_models):
        seed = np.random.randint(0, 1000)  # Random seed for each iteration
        adam_optimizer = Adam(learning_rate=0.001)
        best_weights = []
        # Splitting the dataset
        S_train, S_valid, Y_train, Y_valid = train_test_split(S, Y, test_size=test_size, random_state=seed)

    # Creating the model with the optimizer passed as an argument
        model = make_quad_model(S_train.shape[1], adam_optimizer)

    # Setting up callbacks
        filepath = f'best_quad_AVA_weights_{i}.weights.h5' 
        callbacks = [
                EarlyStopping(patience=patience, min_delta=1e-5, monitor='val_loss', verbose=1),
                ModelCheckpoint(filepath=filepath, save_weights_only=True, save_best_only=True, verbose=1)
        ]

    # Training the model
        H = model.fit(S_train, Y_train, epochs=epochs, callbacks=callbacks, validation_data=(S_valid, Y_valid), verbose=2)
        print(model.summary())
        model.load_weights(filepath)#loads the best weights from the training
        best_weights.append(model.get_weights()) #retreives best weights from load_weights callback
    
        all_best_weights.append(best_weights)

    return all_best_weights

In [ ]:
all_best_weights = run_jackknife_models(S, Y)

In [ ]:
all_best_weights[0][0][0].shape

In [ ]:

symmetrized_0 = (np.array(all_best_weights[0][0][0]) + np.array(all_best_weights[0][0][0]).transpose()) / 2
symmetrized_1 = (np.array(all_best_weights[1][0][0]) + np.array(all_best_weights[1][0][0]).transpose()) / 2
symmetrized_2 = (np.array(all_best_weights[2][0][0]) + np.array(all_best_weights[2][0][0]).transpose()) / 2
symmetrized_3 = (np.array(all_best_weights[3][0][0]) + np.array(all_best_weights[3][0][0]).transpose()) / 2

In [ ]:
f0 = np.matrix.flatten(symmetrized_0)
f1 = np.matrix.flatten(symmetrized_1)
f2 = np.matrix.flatten(symmetrized_2)
f3 = np.matrix.flatten(symmetrized_3)

In [ ]:
A = np.outer(f0, f0.T) + np.outer(f3, f3.T)  + np.outer(f1, f1.T) + np.outer(f2, f2.T)

In [ ]:
A_sym = (A + A.T)/2

In [ ]:
A_eigen = np.linalg.eigh(A_sym)

In [ ]:
eigenvalues = A_eigen[0]

In [ ]:
eigenvalues

In [ ]:
plt.plot(np.sort(np.real(eigenvalues)),  marker = 'o')
plt.show()

In [ ]:
np.dot(A_eigen[1][: , -1], f0)

#these are negative, need to flip the sign of the eigenvector

In [ ]:
eigenvect = A_eigen[1][:, -1]

In [ ]:
eigenvect_0 = np.reshape(eigenvect, (10, 10))

In [ ]:
eigenvect_0_sym = (eigenvect_0 + eigenvect_0.T)/2

In [ ]:
top_eigen_decomp = np.linalg.eig(eigenvect_0_sym)

In [ ]:
top_eigen_decomp

In [ ]:
plt.plot(top_eigen_decomp[1][:, 3])
plt.title('Top Eigenvector AVA Motor')
plt.show()

In [ ]:
plt.plot(top_eigen_decomp[1][:, 3])
plt.title('Top Eigenvector AVA Motor')
plt.gca().invert_xaxis()
plt.show()

# Check Quad Component's Effect

In [ ]:
#calculate variance of weights 
def calculate_dots(J,s):
    dots = []
    for i in range(s.shape[0]):
        dot1 = np.dot(J, s[i])
        dot2 = np.dot(J, s[i].T)
        dots.append(dot2)
    return dots

In [ ]:
J = symmetrized_0 + symmetrized_1 + symmetrized_2 + symmetrized_3

In [ ]:
var = calculate_dots(J = J, s= S)

In [ ]:
np.var(var)

In [ ]:
mean = np.mean(var)
mean

In [ ]:
std = np.std(var)
std

In [ ]:
symmetrized_eigen_0 = np.linalg.eig(symmetrized_0)
symmetrized_eigen_1 = np.linalg.eig(symmetrized_1)
symmetrized_eigen_2 = np.linalg.eig(symmetrized_2)
symmetrized_eigen_3 = np.linalg.eig(symmetrized_3)

In [ ]:
symmetrized_eigen_3[0]

In [ ]:
plt.plot(symmetrized_eigen_3[1][:,1])
plt.show()

In [ ]:
jackknife_0_eigen_0 = np.reshape(symmetrized_eigen_0[1][:, 2], (-1, 1))
jackknife_1_eigen_0 = np.reshape(symmetrized_eigen_1[1][:, 1], (-1, 1))
jackknife_2_eigen_0 = np.reshape(symmetrized_eigen_2[1][:, 3], (-1, 1))
jackknife_3_eigen_0 = np.reshape(symmetrized_eigen_3[1][:, 1], (-1, 1))

In [ ]:
ava_weights = np.concatenate([
    jackknife_0_eigen_0,
    jackknife_1_eigen_0,
    jackknife_2_eigen_0,
    jackknife_3_eigen_0
], axis=1)

In [ ]:
mean_ava = np.average(ava_weights, axis =1)

In [ ]:
ava_weights_df = pd.DataFrame(ava_weights)
ava_weights_df.columns = ['Jackknife 1', 'Jackknife 2', 'Jackknife 3', 'Jackknife 4']

In [ ]:
ava_weights_df['Average'] = mean_ava

In [ ]:
ava_weights_df

In [ ]:
reversed_x = ava_weights_df.index.max() - ava_weights_df.index

# Plotting each column in the DataFrame
for column in ava_weights_df.columns:
    plt.plot(reversed_x, ava_weights_df[column], label=column)

# Adding titles and labels
plt.title('Weights Across Different Models: AVA Sensory Receptive Field')

plt.xlabel('Index')
plt.ylabel('Weights Value')

plt.xticks(reversed_x, labels=ava_weights_df.index)  # Set custom ticks and labels

# Reverse the x-axis so that 0 starts at the origin and increases to the left


# Show the legend and plot
plt.legend(title='Model')
plt.show()

In [ ]:
std_dev = ava_weights_df[['Jackknife 1', 'Jackknife 2', 'Jackknife 3', 'Jackknife 4']].std(axis=1)

In [ ]:
std_dev

In [ ]:
sem = std_dev / np.sqrt(4)

In [ ]:
plt.figure(figsize=(10, 6))
plt.errorbar(x=reversed_x, y=mean_ON, yerr=sem, fmt='-o', label='Average with SEM', ecolor='red', capsize=5, linestyle='-', marker='o', color='blue')
plt.title('ON Receptive Field ')
plt.xlabel('Time')
ticks = ON_weights_df.index
plt.xticks(ticks, labels=ticks[::-1]) 
plt.ylabel('Stimulus')

plt.show()

## average over jackknives by averaging full matrices

In [ ]:
avg_matrix = (symmetrized_0 + symmetrized_1 + symmetrized_2 + symmetrized_3)/4

In [ ]:
#this method doesn't yeild any compex numbers after the eigendecomposition
avg_matrix_decomp = np.linalg.eig(avg_matrix)


In [ ]:
eigenvals = np.sort(avg_matrix_decomp[0])
eigenvals

In [ ]:
plt.plot(eigenvals,  marker = 'o')
plt.show()

## average over jackknives with flattening

In [ ]:
f0 = np.matrix.flatten(symmetrized_0)
f1 = np.matrix.flatten(symmetrized_1)
f2 = np.matrix.flatten(symmetrized_2)
f3 = np.matrix.flatten(symmetrized_3)

In [ ]:
print(np.iscomplex(f0).any())

In [ ]:
f0_outer =  np.outer(f0, f0.T)
print(np.iscomplex(f0_outer).any())

In [ ]:
A = np.outer(f0, f0.T) + np.outer(f1, f1.T) + np.outer(f2, f2.T) + np.outer(f3, f3.T)

In [ ]:
print(np.iscomplex(A).any())

In [ ]:
A_sym = (A + A.T)/2


In [ ]:
A_eigen = np.linalg.eigh(A_sym)


In [ ]:
eigenvalues = A_eigen[0]


In [ ]:
eigenvalues

In [ ]:
plt.plot(np.sort(eigenvalues),  marker = 'o')
plt.show()

In [ ]:
#top most eigenvalue's eigenvector
eigenvect = A_eigen[1][:, -1]

In [ ]:
eigenvect_0 = np.reshape(eigenvect, (10, 10))

In [ ]:
eigenvect_0_sym = (eigenvect_0 + eigenvect_0.T)/2

In [ ]:
eigenvect_0_sym

In [ ]:
top_eigen_decomp = np.linalg.eig(eigenvect_0_sym)

In [ ]:
top_eigen_decomp

In [ ]:
plt.plot(top_eigen_decomp[1][:, 0])
plt.show()

In [ ]:
eigenvect_0_diag = np.linalg.eig(eigenvect_0)

In [ ]:
real_eigenval = np.real(eigenvect_0_diag[0])
real_eigenvect = np.real(eigenvect_0_diag[1])

In [ ]:
real_eigenval

In [ ]:
real_eigenvect 

In [ ]:
plt.plot(real_eigenvect[0],  marker = 'o')
plt.show()

In [ ]:
def calculate_dots(w,s):
    dots = []
    for i in range(s.shape[0]):
        dot = np.dot(w, s[i])
        dots.append(dot)
    return dots

In [ ]:
var = calculate_dots(w = real_eigenvect[0], s= S)

In [ ]:
np.var(var)

In [ ]:
std = np.std(var)
std

In [ ]:
symmetrized_eigen_0 = np.linalg.eig(symmetrized_0)
symmetrized_eigen_1 = np.linalg.eig(symmetrized_1)
symmetrized_eigen_2 = np.linalg.eig(symmetrized_2)
symmetrized_eigen_3 = np.linalg.eig(symmetrized_3)

In [ ]:
symmetrized_eigen_0

In [ ]:
#average all symmertized matrices

In [ ]:
#flatten each symmetrized matrix, do its outer

In [ ]:
#optional, plot the eigenvalues
fig, ax = plt.subplots()
ax.plot(np.sort(symmetrized_eigen_0[0]))

In [ ]:
#eigenvectors and eigenvalues from original
#eigenvalues_0, eigenvectors_0 = np.linalg.eig(all_best_weights[0][0][0])


In [ ]:
#extract eigenvalues from weights from weights matrix
test_eigen_0 = np.linalg.eig(np.matmul(np.array(all_best_weights[0][0][0]), np.array(all_best_weights[0][0][0]).transpose()))
test_eigen_1 = np.linalg.eig(np.matmul(np.array(all_best_weights[1][0][0]), np.array(all_best_weights[1][0][0]).transpose()))
test_eigen_2 = np.linalg.eig(np.matmul(np.array(all_best_weights[2][0][0]), np.array(all_best_weights[2][0][0]).transpose()))
test_eigen_3 = np.linalg.eig(np.matmul(np.array(all_best_weights[3][0][0]), np.array(all_best_weights[3][0][0]).transpose()))


In [ ]:
#reshape's 10th coliumn of eigenvector

eigen_nocomplex_10 = np.reshape(test_eigen_0[1][:, 9], (-1, 1))

In [ ]:
#optional matrix multiplication of eigenvectors
rf_AVA_eigen_10 = np.matmul(eigen_nocomplex_10, eigen_nocomplex_10.T )

In [ ]:
sns.heatmap(rf_AVA_eigen_10)
plt.title("Receptive Field of 10th Eigenvector of AVA Cell 0 in Worm 0, Stim 0")

In [ ]:
sns.heatmap(eigen_nocomplex_10)
plt.title("Receptive Field of 10th Eigenvector of AVA Cell 0 in Worm 0, Stim 0")

In [ ]:
#extract the tenth eigenvector from each jackknife model
jackknife_0_eigen_10 = np.reshape(test_eigen_0[1][:, 9], (-1, 1))
jackknife_1_eigen_10 = np.reshape(test_eigen_1[1][:, 9], (-1, 1))
jackknife_2_eigen_10 = np.reshape(test_eigen_2[1][:, 9], (-1, 1))
jackknife_3_eigen_10 = np.reshape(test_eigen_3[1][:, 9], (-1, 1))

In [ ]:
rf_AVA_eigen_10 = np.matmul(jackknife_0_eigen_10, jackknife_0_eigen_10.T )
rf_AVA_eigen_10 = np.matmul(jackknife_1_eigen_10, jackknife_1_eigen_10.T )
rf_AVA_eigen_10 = np.matmul(jackknife_2_eigen_10, jackknife_2_eigen_10.T )
rf_AVA_eigen_10 = np.matmul(jackknife_3_eigen_10, jackknife_3_eigen_10.T )



In [ ]:
AVA_weights = np.concatenate([
    jackknife_0_eigen_10,
    jackknife_1_eigen_10,
    jackknife_2_eigen_10,
    jackknife_3_eigen_10
], axis=1)

In [ ]:
AVA_weights

In [ ]:
mean_AVA = np.average(AVA_weights, axis =1)

In [ ]:
mean_AVA

In [ ]:
AVA_weights_df = pd.DataFrame(AVA_weights)
AVA_weights_df.columns = ['Jackknife 1', 'Jackknife 2', 'Jackknife 3', 'Jackknife 4']

In [ ]:
AVA_weights_df['Average'] = mean_AVA

In [ ]:
AVA_weights_df

In [ ]:
AVA_weights_df.plot(figsize=(10, 6))  # Adjust the size as necessary

# Adding titles and labels
plt.title('Weights Across Different Models')
plt.xlabel('Index')
plt.ylabel('Weights Value')
plt.grid(True)  # Optional: Adds a grid for better readability

# Show the legend and plot
plt.legend(title='Model')
plt.show()

In [ ]:
std_dev = AVA_weights_df[['Jackknife 1', 'Jackknife 2', 'Jackknife 3', 'Jackknife 4']].std(axis=1)

In [ ]:
sem = std_dev / np.sqrt(4)

In [ ]:
plt.figure(figsize=(10, 6))
plt.errorbar(x=AVA_weights_df.index, y=mean_AVA, yerr=sem, fmt='-o', label='Average with SEM', ecolor='red', capsize=5, linestyle='-', marker='o', color='blue')
plt.title('AVA Receptive Field ')
plt.xlabel('Time')
ticks = AVA_weights_df.index
plt.xticks(ticks, labels=ticks[::-1]) 
plt.ylabel('Stimulus')
plt.show()